# Using the AWRA MS to run a Simulation of the AWRA-L model
There are two ways to run a simulation of the AWRA-L model in the AWRA MS
* Using the On-Demand Simulator
* Using the Server Simulator


## Server Simulation


The Server Simulation is set up very similarly to the On Demand Simulator, except it needs to write the outputs to disk as the model run progresses, as it does not hold the results in memory. This allows the Server Simulator to be the more efficient choice for longer runs over a larger extent. The same output and input maps can be used for either simulator.

This notebook goes through the following steps:

1. Import required libraries
2. Modify the model configuration <br>
 2.1 Read in the default configuration<br>
 2.2 Change forcing data<br>
 2.3 Create model output map and add nodes for saving some model outputs to netcdf files<br>
 2.4 Specify period and extents<br>
3. Put model run specification together<br>
 3.1 Instantiate the simulator<br>
 3.2 Run the model<br>
4. Visualise outputs<br>
 4.1 Import libraries<br>
 4.2 Load model outputs<br>
 4.3 Look at time slice<br>
5. Further simulation functionality<br>
 5.1 Change initial states<br>
 5.2 Fill gaps in forcing with climatology<br>

### 1. Import required libraries

In [8]:
from awrams.simulation import server
from awrams.models.awral.model import AWRALModel, CLIMATE_DATA, FORCING
from awrams.models.settings import TRAINING_DATA_PATH


from awrams.utils import extents
from awrams.utils import datetools

In [3]:
TEST_DATA_PATH = '../../test_data/simulation/'

'/data/cwd_awra_data/AWRACMS/Training/test_data/'

In [2]:
CLIMATE_DATA  = TEST_DATA_PATH

In [10]:
# Instantiate the model

awral = AWRALModel()

### 2. Modify the model configuration

#### 2.1 Read in  default configuration

In [11]:
input_map = awral.get_default_mapping()

In [6]:
input_map

{'init_mleaf_hrusr': transform([2.0, 'sla_hrusr']):{'tfunc': {'name': 'true_divide', 'objtype': 'class', 'module': 'numpy'}, 'func_args': {}}, 'tgrow_hrusr': parameter([]):{'fixed': True, 'max': 1000, 'value': 150.0, 'min': 20}, 'pair': constant([]):{'value': 97500.0}, 'alb_wet_hrudr': parameter([]):{'fixed': True, 'max': 0.5, 'value': 0.16, 'min': 0.1}, 'laimax_hrusr': assign(['lai_max_grid']):{}, 'k_gw_scale': parameter([]):{'fixed': False, 'max': 10, 'value': 0.502237767850745, 'min': 0.1}, 'tmin_f': forcing_from_ncfiles([]):{'cache': False, 'pattern': 'temp_min*', 'path': '/data/cwd_awra_data/AWRACMS/Training/test_data/climate/BOM_climate/temp_min_day/', 'nc_var': 'temp_min_day'}, 'fsoilemax_hrusr': parameter([]):{'fixed': False, 'max': 1, 'value': 0.929708357540083, 'min': 0.2}, 'alb_wet_hrusr': parameter([]):{'fixed': True, 'max': 0.5, 'value': 0.16, 'min': 0.1}, 'ne_scale': parameter([]):{'fixed': False, 'max': 1, 'value': 0.0551891007464962, 'min': 0.01}, 'kssat': transform(('k

#### 2.3 Create model output map and add nodes for saving some model outputs to netcdf files

In [7]:
# These are the default model outpus
# OUTPUTS_AVG : Weighted average of HRU outputs
# OUTPUTS_CELL : Cell level outputs (ie processes happening outside of HRU; groundwater store etc)
# OUTPUTS_HRU : HRU level outputs (ie separate output per HRU)

awral.OUTPUTS

{'OUTPUTS_AVG': ['e0', 'etot', 'dd', 's0', 'ss', 'sd'],
 'OUTPUTS_CELL': ['qtot', 'sr', 'sg'],
 'OUTPUTS_HRU': ['s0', 'ss', 'sd', 'mleaf']}

In [8]:
# Get the output map
# This gives us a (nodegraph) mapping for model outputs
# By default these are purely in-memory; we can add file outputs in the next step

omap = awral.get_output_mapping()

omap

{'dd': output_variable([]):{'var_name': 'dd'},
 'e0': output_variable([]):{'var_name': 'e0'},
 'etot': output_variable([]):{'var_name': 'etot'},
 'mleaf_hrudr': output_variable([]):{'var_name': 'mleaf_hrudr'},
 'mleaf_hrusr': output_variable([]):{'var_name': 'mleaf_hrusr'},
 'qtot': output_variable([]):{'var_name': 'qtot'},
 's0': output_variable([]):{'var_name': 's0'},
 's0_hrudr': output_variable([]):{'var_name': 's0_hrudr'},
 's0_hrusr': output_variable([]):{'var_name': 's0_hrusr'},
 'sd': output_variable([]):{'var_name': 'sd'},
 'sd_hrudr': output_variable([]):{'var_name': 'sd_hrudr'},
 'sd_hrusr': output_variable([]):{'var_name': 'sd_hrusr'},
 'sg': output_variable([]):{'var_name': 'sg'},
 'sr': output_variable([]):{'var_name': 'sr'},
 'ss': output_variable([]):{'var_name': 'ss'},
 'ss_hrudr': output_variable([]):{'var_name': 'ss_hrudr'},
 'ss_hrusr': output_variable([]):{'var_name': 'ss_hrusr'}}

In [22]:
# Add some file output nodes to our mapping

def build_output_mapping(output_map, outpath):
    from awrams.utils.nodegraph import nodes

    #outpath = './_results_ServerSim/'
    
    FILE_OUTPUT_VARS = ['ss','sd','qtot','etot']
    
    for f in FILE_OUTPUT_VARS:
            output_map[f+'_ncsave'] = nodes.write_to_annual_ncfile(outpath,f)

    return output_map


In [23]:
outpath = './_results_ServerSim/'
omap = build_output_mapping(omap, outpath)

# The updated output map contains write_to_annual_ncfile nodes for the variables we specified above

omap

{'dd': output_variable([]):{'var_name': 'dd'},
 'e0': output_variable([]):{'var_name': 'e0'},
 'etot': output_variable([]):{'var_name': 'etot'},
 'etot_ncsave': write_to_annual_ncfile(['etot']):{'path': './_results_ServerSim/', 'nc_var': 'etot', 'mode': 'a'},
 'mleaf_hrudr': output_variable([]):{'var_name': 'mleaf_hrudr'},
 'mleaf_hrusr': output_variable([]):{'var_name': 'mleaf_hrusr'},
 'qtot': output_variable([]):{'var_name': 'qtot'},
 'qtot_ncsave': write_to_annual_ncfile(['qtot']):{'path': './_results_ServerSim/', 'nc_var': 'qtot', 'mode': 'a'},
 's0': output_variable([]):{'var_name': 's0'},
 's0_hrudr': output_variable([]):{'var_name': 's0_hrudr'},
 's0_hrusr': output_variable([]):{'var_name': 's0_hrusr'},
 'sd': output_variable([]):{'var_name': 'sd'},
 'sd_hrudr': output_variable([]):{'var_name': 'sd_hrudr'},
 'sd_hrusr': output_variable([]):{'var_name': 'sd_hrusr'},
 'sd_ncsave': write_to_annual_ncfile(['sd']):{'path': './_results_ServerSim/', 'nc_var': 'sd', 'mode': 'a'},
 'sg'

#### 2.4	Define the required period and spatial extent
Because it is the Server Simulator the extent can be defined as the full continent which is the default, 
but for a longer period that one day

In [11]:
period = datetools.dates('dec 2010 - jan 2011')

edef = extents.get_default_extent()


### 3. Put the model run specification together 

#### 3.1 Instantiate the server simulator

In [12]:
sim = server.SimulationServer(awral)

#### 3.2 Run the simulation

In [13]:
sim.run(input_map,omap,period,edef)

2017-11-15 03:43:58,547 INFO Getting I/O dataspecs...
2017-11-15 03:43:59,616 INFO Initialising output files...
2017-11-15 03:44:00,066 INFO Building buffers...
2017-11-15 03:44:01,420 INFO Running simulation...
2017-11-15 03:44:03,491 INFO completed 5.06%
2017-11-15 03:44:04,424 INFO completed 10.12%
2017-11-15 03:44:05,299 INFO completed 15.18%
2017-11-15 03:44:06,403 INFO completed 20.24%
2017-11-15 03:44:07,287 INFO completed 25.30%
2017-11-15 03:44:08,182 INFO completed 30.36%
2017-11-15 03:44:09,062 INFO completed 35.42%
2017-11-15 03:44:10,330 INFO completed 40.48%
2017-11-15 03:44:11,383 INFO completed 45.54%
2017-11-15 03:44:12,241 INFO completed 50.60%
2017-11-15 03:44:13,493 INFO completed 55.65%
2017-11-15 03:44:14,642 INFO completed 60.71%
2017-11-15 03:44:15,513 INFO completed 65.77%
2017-11-15 03:44:16,424 INFO completed 70.83%
2017-11-15 03:44:17,353 INFO completed 75.89%
2017-11-15 03:44:18,509 INFO completed 80.95%
2017-11-15 03:44:19,414 INFO completed 86.01%
2017-11

### 4. Visualise outputs

Because ouputs are written out, we can use the process described in the [Visualisation] notebook

[Visualisation]: ../Visualisation/Visualisation.ipynb

#### 4.1 Import visualisation libraries

In [14]:
import awrams.visualisation.vis as vis
import awrams.visualisation.results as res

import awrams.utils.extents as extents


#### 4.2 Load model outputs

In [15]:
results = res.load_results('./_results_ServerSim')

In [16]:
results.variables

OrderedDict([('etot', etot), ('ss', ss), ('sd', sd), ('qtot', qtot)])

#### 4.3 Look at a time slice

In [17]:
results[:,'1 jan 2011',:].spatial()

/data/cwd_awra_data/AWRACMS/MINICONDA3/envs/awra-cms/lib/python3.4/site-packages/numpy/ma/core.py:852: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)


### 5. Further simulation functionality

These will apply equally to the OnDemand and Server options

#### 5.1 How to change the initial states


In [5]:
# Get the default initial states path for the supplied training data

from awrams.models.awral.settings import INITIAL_STATES_PATH 

INITIAL_STATES_PATH 

'/data/cwd_awra_data/AWRACMS/Training/test_data/simulation/initial_states/'

In [19]:
# Loading initial states from netCDF files, a fixed set of initial states independent of date
# These variables you have in your file can have any name, these are from a previous model version so have slightly different variable names 

def initial_states_from_files(imap):
    from awrams.utils.nodegraph import nodes

    data_path = INITIAL_STATES_PATH

    mapping = imap.copy()

    mapping['init_sr'] = nodes.init_state_from_ncfile(data_path,'sr_bal*','sr_bal')
    mapping['init_sg'] = nodes.init_state_from_ncfile(data_path,'sg_bal*','sg_bal')

    HRU = {'_hrusr':'_sr','_hrudr':'_dr'} # Conversion table for old->new state naming conventions
    for hru in ('_hrusr','_hrudr'):
        for state in ["s0","ss","sd",'mleaf']:
            mapping['init_'+state+hru] = nodes.init_state_from_ncfile(data_path,state+HRU[hru]+'*',state+HRU[hru])
            
    return mapping


In [20]:
input_map = initial_states_from_files(input_map)

# Examine the new init_states nodes...

dict([(k,v) for k,v in input_map.items() if k.startswith('init')])

{'init_mleaf_hrudr': init_state_from_ncfile([]):{'pattern': 'mleaf_dr*', 'path': '/data/cwd_awra_data/AWRACMS/Training/test_data/simulation/initial_states/', 'nc_var': 'mleaf_dr'},
 'init_mleaf_hrusr': init_state_from_ncfile([]):{'pattern': 'mleaf_sr*', 'path': '/data/cwd_awra_data/AWRACMS/Training/test_data/simulation/initial_states/', 'nc_var': 'mleaf_sr'},
 'init_s0_hrudr': init_state_from_ncfile([]):{'pattern': 's0_dr*', 'path': '/data/cwd_awra_data/AWRACMS/Training/test_data/simulation/initial_states/', 'nc_var': 's0_dr'},
 'init_s0_hrusr': init_state_from_ncfile([]):{'pattern': 's0_sr*', 'path': '/data/cwd_awra_data/AWRACMS/Training/test_data/simulation/initial_states/', 'nc_var': 's0_sr'},
 'init_sd_hrudr': init_state_from_ncfile([]):{'pattern': 'sd_dr*', 'path': '/data/cwd_awra_data/AWRACMS/Training/test_data/simulation/initial_states/', 'nc_var': 'sd_dr'},
 'init_sd_hrusr': init_state_from_ncfile([]):{'pattern': 'sd_sr*', 'path': '/data/cwd_awra_data/AWRACMS/Training/test_data

In [24]:
sim = server.SimulationServer(awral)

outpath = './_results_ServerSim_initstates/'  # set a new output path or clear previous outputs. Errors are likely if files exist.
omap = build_output_mapping(omap, outpath)

sim.run(input_map,omap,period,edef)

2017-11-15 03:51:10,131 INFO Getting I/O dataspecs...
2017-11-15 03:51:11,999 INFO Initialising output files...
2017-11-15 03:51:12,372 INFO Building buffers...
2017-11-15 03:51:12,653 INFO Running simulation...
2017-11-15 03:51:15,595 INFO completed 5.06%
2017-11-15 03:51:16,589 INFO completed 10.12%
2017-11-15 03:51:17,619 INFO completed 15.18%
2017-11-15 03:51:19,135 INFO completed 20.24%
2017-11-15 03:51:20,167 INFO completed 25.30%
2017-11-15 03:51:21,249 INFO completed 30.36%
2017-11-15 03:51:21,998 INFO completed 35.42%
2017-11-15 03:51:23,676 INFO completed 40.48%
2017-11-15 03:51:24,889 INFO completed 45.54%
2017-11-15 03:51:25,793 INFO completed 50.60%
2017-11-15 03:51:27,390 INFO completed 55.65%
2017-11-15 03:51:28,599 INFO completed 60.71%
2017-11-15 03:51:29,413 INFO completed 65.77%
2017-11-15 03:51:30,347 INFO completed 70.83%
2017-11-15 03:51:31,495 INFO completed 75.89%
2017-11-15 03:51:32,947 INFO completed 80.95%
2017-11-15 03:51:33,861 INFO completed 86.01%
2017-11

In [26]:
# Using the same data as above, but this time load the states into memory, and store in a python dictionary
# Use this when you want to compute custom states, for example

def initial_states_from_dict(imap,period,extent):
    from awrams.utils.io.data_mapping import SplitFileManager
    from awrams.utils.nodegraph import nodes
    
    mapping = imap.copy()

    data_path = INITIAL_STATES_PATH

    node_names = {'mleaf_dr': 'init_mleaf_hrudr',
                  'mleaf_sr': 'init_mleaf_hrusr',
                  's0_dr': 'init_s0_hrudr',
                  's0_sr': 'init_s0_hrusr',
                  'ss_dr': 'init_ss_hrudr',
                  'ss_sr': 'init_ss_hrusr',
                  'sd_dr': 'init_sd_hrudr',
                  'sd_sr': 'init_sd_hrusr',
                  'sg_bal': 'init_sg',
                  'sr_bal': 'init_sr'}

    data_map = {}
    period = [period[0] - 1] # Use states from the previous day
    
    for k in node_names:
        sfm = SplitFileManager.open_existing(data_path,k+'*nc',k)
        data_map[node_names[k]] = sfm.get_data(period,extent)[0]  # At this point you could manipulate the initial states data
        
    # nodes.init_states_from_dict is a convenience function to update a mapping in-place
    nodes.init_states_from_dict(mapping,data_map,extent)
    
    return mapping


In [27]:
input_map = initial_states_from_dict(input_map,period,edef)

In [28]:
sim = server.SimulationServer(awral)

outpath = './_results_ServerSim_initstatesfromdate/'
omap = build_output_mapping(omap, outpath)

sim.run(input_map,omap,period,edef)

2017-11-15 04:05:46,636 INFO Getting I/O dataspecs...
2017-11-15 04:05:47,832 INFO Initialising output files...
2017-11-15 04:05:48,209 INFO Building buffers...
2017-11-15 04:05:48,500 INFO Running simulation...
2017-11-15 04:05:50,648 INFO completed 5.06%
2017-11-15 04:05:51,556 INFO completed 10.12%
2017-11-15 04:05:52,354 INFO completed 15.18%
2017-11-15 04:05:53,563 INFO completed 20.24%
2017-11-15 04:05:54,432 INFO completed 25.30%
2017-11-15 04:05:55,333 INFO completed 30.36%
2017-11-15 04:05:56,075 INFO completed 35.42%
2017-11-15 04:05:57,419 INFO completed 40.48%
2017-11-15 04:05:58,471 INFO completed 45.54%
2017-11-15 04:05:59,234 INFO completed 50.60%
2017-11-15 04:06:00,482 INFO completed 55.65%
2017-11-15 04:06:01,556 INFO completed 60.71%
2017-11-15 04:06:02,292 INFO completed 65.77%
2017-11-15 04:06:03,051 INFO completed 70.83%
2017-11-15 04:06:03,884 INFO completed 75.89%
2017-11-15 04:06:05,045 INFO completed 80.95%
2017-11-15 04:06:05,848 INFO completed 86.01%
2017-11

#### 5.2 Fill gaps in forcing data with climatology

Example infilling of solar radiation using monthly climatology

In [34]:
def insert_solar_climatology(imap):
    from awrams.utils.nodegraph import nodes
    # from awrams.models.settings import CLIMATOLOGIES
    CLIMATOLOGIES = {'solar': (TRAINING_DATA_PATH+'simulation/climatology/Rad_1990_2009.nc','solar_exposure_day')}

    
    imap = imap.copy()
    
    imap['solar_f_orig'] = imap['solar_f'] #'Move' the forcing node to a new name
    imap['solar_climatology_f'] = nodes.monthly_climatology(*CLIMATOLOGIES['solar']) # Loads monthly climatology from default file

    # Replace 'solar_f' with infilled data
    # This ensures that any other nodes in the graph who use solar_f as input will automatically receive the infilled data
    # nodes.gap_filler takes the first argument as 'gappy' data, and in infills with data from the second argument
    imap['solar_f']  = nodes.gap_filler('solar_f_orig','solar_climatology_f')
    
    return imap


In [35]:
input_map = insert_solar_climatology(input_map)

In [36]:
input_map['solar_f']

gap_filler(['solar_f_orig', 'solar_climatology_f']):{}

In [37]:
sim = server.SimulationServer(awral)

outpath = './_results_ServerSim_solarclim/'
omap = build_output_mapping(omap, outpath)

sim.run(input_map,omap,period,edef)

2017-11-15 04:10:47,408 INFO Getting I/O dataspecs...
2017-11-15 04:10:49,217 INFO Initialising output files...
2017-11-15 04:10:49,543 INFO Building buffers...
2017-11-15 04:10:49,818 INFO Running simulation...
2017-11-15 04:10:52,356 INFO completed 5.06%
2017-11-15 04:10:53,289 INFO completed 10.12%
2017-11-15 04:10:54,140 INFO completed 15.18%
2017-11-15 04:10:55,428 INFO completed 20.24%
2017-11-15 04:10:56,339 INFO completed 25.30%
2017-11-15 04:10:57,252 INFO completed 30.36%
2017-11-15 04:10:58,008 INFO completed 35.42%
2017-11-15 04:10:59,451 INFO completed 40.48%
2017-11-15 04:11:00,530 INFO completed 45.54%
2017-11-15 04:11:01,321 INFO completed 50.60%
2017-11-15 04:11:02,690 INFO completed 55.65%
2017-11-15 04:11:03,860 INFO completed 60.71%
2017-11-15 04:11:04,605 INFO completed 65.77%
2017-11-15 04:11:05,369 INFO completed 70.83%
2017-11-15 04:11:06,323 INFO completed 75.89%
2017-11-15 04:11:07,577 INFO completed 80.95%
2017-11-15 04:11:08,443 INFO completed 86.01%
2017-11